# Numerical estimate of the optimal CZ gate in hybrid encoding in the KLM model

The target operation in the Fock basis is one that brings $|0010>|01>$ into $-|0010>|01>$, $|0001>|01>$ into $-|0001>|01>$ and acts as the identity on the rest. The operator that describes this transformation is the target $A_t$.

The problem is framed as an optimization problem on the unitary matrix $\Lambda$ which determines the transformation of the bosonic modes. $\Lambda$ has a representation on the space of Fock states $U(\Lambda)$. $U$ acts on the computational states as a set of Kraus operators, of which the first one $A:=K_0$ is of importance as it describes the state obtained after postselection on the ancilla qubits.

The solution consists on optimizing $\Lambda$ on the cost function of $1-F$ where $F$ is the fidelity as defined in [Uskov et al.].

The presence of the hybrid encoding changes the form of the target $A_t$, while the optimization problem remains the same and can be solved as in the previous problem.

## Representation of the unitary on the Fock states

An implementation of the procedure that computes the representation $U$ on the Fock states is given, following [Schee, 2004]

In [1]:
import numpy as np
from math import comb, factorial
from itertools import combinations_with_replacement
from itertools import permutations

def factl(l):
	return [factorial(i) for i in l]

def permanent(matrix):
    n = len(matrix)
    perm_sum = 0

    for perm in permutations(range(n)):
        product = 1
        for i in range(n):
            product *= matrix[i, perm[i]]
        perm_sum += product	

    return perm_sum

def get_A(lam):
	N = lam.shape[0]
	n = 2
	N_out = comb(N+n-1, n)
	A = np.ones((N_out, N_out))*42

	for idx_n, comb_n in enumerate(combinations_with_replacement(range(N), n)):
		for idx_m, comb_m in enumerate(combinations_with_replacement(range(N), n)):
			n_i = np.zeros(N, dtype=int)
			for x in comb_n:
				n_i[x] += 1
			
			m_i = np.zeros(N, dtype=int)
			for x in comb_m:
				m_i[x] += 1

			omega = []
			for i,x in enumerate(n_i):
				omega.extend([i]*x)
			omega = np.array(omega)

			omegap = []
			for i,x in enumerate(m_i):
				omegap.extend([i]*x)
			omegap = np.array(omegap)

			sub = lam[omegap]
			sub = sub[:, omega]
			prefactor = np.sqrt(np.prod(factl(omega)) * np.prod(factl(omegap)))
			A[idx_m, idx_n] =  prefactor * permanent(sub)
	return A

In [2]:
# example:
lam = np.random.random((3,3))*10
print(lam)
get_A(lam)

[[9.18614442 9.08553164 4.27081625]
 [1.40547656 8.56390438 1.85138264]
 [8.88437065 1.30432526 0.27441982]]


array([[168.7704985 , 166.92201143, 110.96580018, 165.09377021,
        109.75042871,  72.95948592],
       [ 25.82182138,  91.43876417,  32.54048899, 155.6152483 ,
         75.51286306,  31.62766023],
       [230.83673916, 131.09894188,  80.92874927,  33.51824298,
         16.12756694,   6.62981403],
       [  3.95072874,  24.0727338 ,   7.35977925, 146.68091639,
         44.84489271,  13.71047067],
       [ 35.31793234, 110.19283272,  33.66812034,  31.59386146,
          9.52982044,   2.87399922],
       [315.72816706,  46.35243637,  13.79167861,   6.80505758,
          2.02477312,   0.60244989]])

## Target matrix

In [3]:
N = 6 # six photonic modes
n = 2 # two photons
N_fock = comb(N+n-1, n)
At = np.identity(N_fock)

def index_of_fock_state(N, n, v):
	v = np.array(v)
	i = 0
	for c in combinations_with_replacement(range(N), n):
		fock_s = np.zeros(N, dtype=int)
		for x in c:
			fock_s[x] += 1
		if np.all(np.array(fock_s) == v):
			return i
		i += 1

fock_state = [0,0,1,0,0,1]
idx = index_of_fock_state(N, n, fock_state)
At[idx, idx] = -1

fock_state = [0,0,0,1,0,1]
idx = index_of_fock_state(N, n, fock_state)
At[idx, idx] = -1

# Columns outside of the input space allowed by the encoding can be dropped to obtain a representation in the form of [Uskov et al.]
def drop_invalid_input(M):
	allowed_input_space = np.array([
		[1,0,0,0,1,0],
		[0,1,0,0,1,0],
		[0,0,1,0,1,0],
		[0,0,0,1,1,0],
		[1,0,0,0,0,1],
		[0,1,0,0,0,1],
		[0,0,1,0,0,1],
		[0,0,0,1,0,1]
	])
	allowed_idx = []
	for state in allowed_input_space:
		idx = index_of_fock_state(N, n, state)
		allowed_idx.append(idx)
	return M[:, allowed_idx]

At = drop_invalid_input(At)

In [4]:
print(At)

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


## Optimization problem

The optimization problem is solved as in the first problem

In [5]:
import numpy as np
from scipy.optimize import minimize

def inner_product(A, B, Mc=4):
	return np.trace(A.conj().T @ B) / 2**Mc

def cost_function(Lambda):
	return 1 - F(Lambda)

def cost_function_flattened(flat_Lambda):
	Lambda = flat_Lambda.reshape((N, N))
	return cost_function(Lambda)

def F(Lambda):
	A = get_A(Lambda)
	return inner_product(At, A) * inner_product(A, At) / (inner_product(At, At) * inner_product(A, A))

# Define the optimization problem
N = 6  # Size of the unitary matrix
initial_guess = np.eye(N).flatten()  # Initial guess for the candidate matrix

# Solve the optimization problem
result = minimize(cost_function_flattened, initial_guess, method='CG')

# Get the optimized candidate matrix
optimized_matrix = result.x.reshape((N, N))

# Print the optimized candidate matrix
print(optimized_matrix)


: 